In [3]:
import pandas as pd
import numpy as np
import requests as rq
import re
import bs4
from tqdm import tqdm
from langdetect import detect
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import cross_val_score ,StratifiedKFold,GridSearchCV,RandomizedSearchCV
from sklearn.pipeline import Pipeline
import nltk
from sklearn.decomposition import TruncatedSVD


#### Сбор данных 

Получение адресов

In [4]:
arr_link = []

url = "https://www.e-katalog.ru"
for i in tqdm(range(0,20)):
    adres = "https://www.e-katalog.ru/list/122/" + str(i) + "/"
    html = rq.get(adres)
    parser = bs4.BeautifulSoup(html.text,"lxml", fromEncoding="cp1251")
    parser = parser.findAll("div", attrs={"class" : "ib model-short-links no-mobile"})

    for case in parser:
        com = case.find(text=re.compile("Отзывы"))
        if com is None:
            continue
        c = com.findParent().findParent()
        link =  url + c.attrs["link"]
        arr_link.append(link)

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]C:\Users\Grigorii\anaconda3\lib\site-packages\bs4\__init__.py:205: UserWarning: The "fromEncoding" argument to the BeautifulSoup constructor has been renamed to "from_encoding."
  warnings.warn(
C:\Users\Grigorii\anaconda3\lib\site-packages\bs4\__init__.py:220: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")
100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:30<00:00,  1.53s/it]


In [5]:
arr_link[:5]

['https://www.e-katalog.ru/review/xiaomi-redmi-note-8-pro-128gb-6gb/',
 'https://www.e-katalog.ru/review/xiaomi-redmi-note-9-pro-128gb/',
 'https://www.e-katalog.ru/review/apple-iphone-xr-64gb/',
 'https://www.e-katalog.ru/review/apple-iphone-x-64gb/',
 'https://www.e-katalog.ru/review/samsung-galaxy-s10-128gb/']

Получим отзывы

In [6]:
%%time
plus, minus = [], []
for l in tqdm(arr_link):
    html = rq.get(l)
    parser = bs4.BeautifulSoup(html.text,"lxml", fromEncoding="cp1251")
    arr_plus = parser.findAll("div", attrs={"class" : "review-plus"})
    arr_minus = parser.findAll("div", attrs={"class" : "review-minus"})
    for x in arr_plus:
        # выбросим все наблюдения где мало слов
        if len(x.text) < 105:
            continue
        # выбросим комментраии не на русском языке
        lang = detect(x.text)
        if lang != "ru":
            continue
        plus.append(x.text)
    # всё толже самое для отрицательных отзывов
    for x in arr_minus:
        if len(x.text) < 105:
            continue
        lang = detect(x.text)
        if lang != "ru":
            continue
        minus.append(x.text)
    
    

100%|████████████████████████████████████████████████████████████████████████████████| 389/389 [05:47<00:00,  1.12it/s]

Wall time: 5min 47s


In [7]:
len(minus)

785

In [8]:
len(plus)

915

In [9]:
minus[:5]

['Присутствие нагрева в тяжёлых играх на ультра настройках. Снижение настроек до средних снимает нагрузку и температура падает до комфортных.',
 'Есть глюки в некоторых приложениях, но я думаю это поправят.\rСильно выпирает камера, стандартный чехол от царапин на камере не защитит, лучше сразу нормальный купить.',
 'Сырое ПО, надеюсь как и во всех xiaomi со временем допилят. Звонилка ужасная (особенно после того как гугл запретил записывать звонки) всегда пользовался (на редми 3, редми нот 3 про) звонилкой true phone. Очень расширяет возможности и удобства звонили по сравнению со стандартной, но здесь программа работает не корректно (скорее виноват андроид 10) надеюсь со временем допилят. Записи звонков нет (гугл запретил), а мне эта функция очень нужна т. к.  по телефону передаётся много инфы, а ручки и бумаги что бы всё записать под рукой нет (потому как большую часть времени за рулём). Думал установить как на раннии аппараты стороннее приложение cube acr (очень хорошее функционально

In [10]:
texts = plus + minus
labels = [0] * len(plus) + [1] * len(minus)

#### Построение базовой модели

In [12]:
def make_pipeline(vectorizer, transformer, classifier):
    return Pipeline([
            ('vectorizer', vectorizer),
            ('transformer', transformer),
            ('classifier', classifier)
        ])

In [13]:
pipe_base = make_pipeline(CountVectorizer(), TfidfTransformer(),LogisticRegression(solver = "liblinear"))
result_base = cross_val_score(pipe_base, texts, labels, cv =5, scoring="accuracy")
print(result_base.mean())

0.8992028266099817


#### Считаем тестовые данные

In [14]:
rock = open('test.csv', 'r', encoding="utf8")
r = rock.read().replace('\n', ' ')

parser = bs4.BeautifulSoup(r,"lxml")

cases = parser.find_all("review")

test_arr = []
for i in cases:
    test_arr.append(i.text)

In [15]:
test_arr[50]

'Ужасно глючит, установка не предустановленных тем грозит зависанием везде и во всём, может посто зависнуть во время разговора, синхронизации никакой, не советую заходить в почту через быструю клавишу просто сваливает настройки интернета которые потом смогли переустановить только после пере прошивки, громкость динамика для разговора очень тихая, о-о-очень ме-е-едленный!!! Телефон крепкий с хорошим аккумулятором, но слишком сырой для использования, на улице собеседника почти не слышно, при попытке увеличить громкость телефон направляет на приём к ухо горло носу:)) да и в игры в нём полноценно поиграть не получится клавиатура отключается, это далеко не первая нокия в моих руках, но это нечто. Как то поросился обновится браузер...да...пи...пи...ПИ....- пере прошивка. За время его использования полностью в нём разочаровался. '

#### Выбор оптимальной модели

In [16]:
#конструктор ценочной функции
def make_estimator(classifier, params_grid, data, labels):
    pipeline = make_pipeline(CountVectorizer(), TfidfTransformer(), classifier)
    grid_cv = RandomizedSearchCV(pipeline, params_grid, scoring='accuracy', cv=5, random_state=12, n_iter=100,n_jobs=-1)
    grid_cv.fit(data, labels)
    return grid_cv


In [17]:
# Параметры для каждой модели

params_grid_lr = {
    'classifier__C': np.arange(0.1, 2, 0.1),
    'classifier__max_iter': np.arange(50, 500, 50),
    'classifier__solver': ['lbfgs', 'liblinear', 'sag']
}
params_grid_lsvc = {
    'classifier__loss': ['hinge', 'squared_hinge'], 
    'classifier__max_iter': np.arange(100, 1000, 50),
    'classifier__tol': [1e-5, 1e-4, 1e-3],
    'classifier__C': np.arange(0.1, 2, 0.1)
}
params_grid_sgdc = {
    'classifier__loss': ['log', 'hinge', 'modified_huber'], 
    'classifier__penalty':  ['l1', 'l2', 'elasticnet'], 
    'classifier__max_iter': np.arange(100, 1000, 50),
    'classifier__tol': np.arange(1e-5, 1e-3, 1e-5),
}

# Параетры векторизации
params_grid_vectorizer = {
    'vectorizer__max_df' : [0.85, 0.9, 0.95, 1.0],
    'vectorizer__min_df' : [1, 10, 20], 
    'vectorizer__ngram_range' : [(1, x) for x in range(1,7)]
}

In [18]:
%%time
grid_search_lr = make_estimator(LogisticRegression(), 
                                {**params_grid_vectorizer, **params_grid_lr}, texts, labels)
print("LogisticRegression: %.4f" % grid_search_lr.best_score_ )


LogisticRegression: 0.9013
Wall time: 1min 40s


In [19]:

%%time
grid_search_lsvc = make_estimator(LinearSVC(random_state=12), 
                                  {**params_grid_vectorizer, **params_grid_lsvc}, texts, labels)
print("LinearSVC: %.4f" % grid_search_lsvc.best_score_ )

LinearSVC: 0.9149
Wall time: 1min 15s


In [20]:

%%time
grid_search_sgdc = make_estimator(SGDClassifier(random_state=12), 
                                  {**params_grid_vectorizer, **params_grid_sgdc},  texts, labels)
print("SGDClassifier: %.4f" % grid_search_sgdc.best_score_ )


SGDClassifier: 0.9091
Wall time: 1min 16s


In [21]:
prediction = grid_search_lsvc.best_estimator_.predict(test_arr)

In [22]:
df = pd.DataFrame()
df["Id"] = [ x for x in range(0,100)]
df["y"] = prediction
df["y"] = df["y"].map({0:"pos", 1:"neg"})
df.to_csv("submission.csv", index = False)

#### Результат

Так как соревнование уже завершено, прикрипляю скриншот с последним результатом 

!["Результат"](sentiment-kaggle.png)

#### Для приложения

In [23]:
import pickle

In [24]:
pickle.dump(grid_search_lsvc.best_estimator_, open("model.pkl", "wb") )